In [2]:
import ccxt
import my_config
import schedule
import pandas as pd
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')

import numpy as np
from datetime import datetime
import time

#Elimanates time/order issues
ccxt.binanceus({ 'options':{ 'adjustForTimeDifference':True}})

exchange = ccxt.binanceus({
"apiKey": my_config.BINANCE_KEY,
"secret": my_config.BINANCE_SECRET,
'enableRateLimit': True})

In [5]:
#SuperTrends Formula
def tr(data):
    data['previous_close'] = data['close'].shift(1)
    data['high-low'] = abs(data['high'] - data['low'])
    data['high-pc'] = abs(data['high'] - data['previous_close'])
    data['low-pc'] = abs(data['low'] - data['previous_close'])

    tr = data[['high-low', 'high-pc', 'low-pc']].max(axis=1)

    return tr

def atr(data, period):
    data['tr'] = tr(data)
    atr = data['tr'].rolling(period).mean()

    return atr

def supertrend(df, period=7, atr_multiplier=3):
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = atr(df, period)
    df['upperband'] = hl2 + (atr_multiplier * df['atr'])
    df['lowerband'] = hl2 - (atr_multiplier * df['atr'])
    df['in_uptrend'] = True

    for current in range(1, len(df.index)):
        previous = current - 1

        if df['close'][current] > df['upperband'][previous]:
            df['in_uptrend'][current] = True
        elif df['close'][current] < df['lowerband'][previous]:
            df['in_uptrend'][current] = False
        else:
            df['in_uptrend'][current] = df['in_uptrend'][previous]

            if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous]

            if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous]
        
    return df

in_position = False
ticker = 'VTHO/USD'
bar = exchange.fetch_ohlcv(f'{ticker}', timeframe='1m', limit=5)
order_size = int(11/bar[4][1]-(.05*(11/bar[4][1]))) #$12_min_notional

def check_buy_sell_signals(df):
    global in_position,order_size,ticker

    print("Analyzing incoming data...")
    print(df.tail(3))
    last_row_index = len(df.index) - 1
    previous_row_index = last_row_index - 1

    if not df['in_uptrend'][previous_row_index] and df['in_uptrend'][last_row_index]:
        print("Changed to uptrend - Buy")
        
        if not in_position:

#Buy order  #order = exchange.create_market_buy_order('VTHO/USD',order_size)
            #print('Status:'+order['info']['status'],
            #      'Price:'+order['trades'][0]['info']['price'],
            #      'Quantity:'+order['info']['executedQty'],
            #      'Type:'+order['info']['side'])
            
            fake_order = exchange.fetch_ohlcv(f'{ticker}', timeframe='1m', limit=1)#
            print("Fake-buy here:",fake_order)
            in_position = True
        else:
            print("Already in position, no task.")
    
    if df['in_uptrend'][previous_row_index] and not df['in_uptrend'][last_row_index]:
        if in_position: #If in position && previous purchase price << current/market price.
            print("Changed to downtrend - Sell")

#Sell order #order = exchange.create_market_sell_order('VTHO/USD',order_size)
            #print('Status:'+order['info']['status'],
            #      'Price:'+order['trades'][0]['info']['price'],
            #      'Quantity:'+order['info']['executedQty'],
            #      'Type:'+order['info']['side'])
            #in_position = False
            
            fake_order = exchange.fetch_ohlcv(f'{ticker}', timeframe='1m', limit=1)#
            print("Fake-sell here:",fake_order)
        else:
            print("No selling position, no task.")

In [7]:
def run_bot():
    print(f"\nFetching new bars for {datetime.now().isoformat()}")
    bars = exchange.fetch_ohlcv(f'{ticker}', timeframe='1m', limit=25)
    df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    supertrend_data = supertrend(df)
    
    check_buy_sell_signals(supertrend_data)
    print()


schedule.every(15).seconds.do(run_bot)


while True:
    schedule.run_pending()
    time.sleep(1)

Fetching new bars for 2021-09-13T08:30:20.748899
Analyzing incoming data...
             timestamp    open    high     low   close     volume  \
21 2021-09-13 15:27:00  0.0094  0.0095  0.0094  0.0095   114057.0   
22 2021-09-13 15:28:00  0.0095  0.0095  0.0094  0.0094  3873921.0   
23 2021-09-13 15:29:00  0.0096  0.0096  0.0095  0.0095    51149.0   

    previous_close  high-low  high-pc  low-pc      tr       atr  upperband  \
21          0.0095    0.0001   0.0000  0.0001  0.0001  0.000029   0.009536   
22          0.0095    0.0001   0.0000  0.0001  0.0001  0.000043   0.009579   
23          0.0094    0.0001   0.0002  0.0001  0.0002  0.000057   0.009579   

    lowerband  in_uptrend  
21   0.009457        True  
22   0.009321       False  
23   0.009379       False  

Fetching new bars for 2021-09-13T08:30:36.055126
Analyzing incoming data...
             timestamp    open    high     low   close     volume  \
21 2021-09-13 15:27:00  0.0094  0.0095  0.0094  0.0095   114057.0   
22 2021

Fetching new bars for 2021-09-13T08:32:23.243170
Analyzing incoming data...
             timestamp    open    high     low   close   volume  \
21 2021-09-13 15:29:00  0.0096  0.0096  0.0095  0.0095  51149.0   
22 2021-09-13 15:30:00  0.0096  0.0096  0.0096  0.0096  13477.0   
23 2021-09-13 15:31:00  0.0096  0.0096  0.0096  0.0096  26954.0   

    previous_close  high-low  high-pc  low-pc      tr       atr  upperband  \
21          0.0094    0.0001   0.0002  0.0001  0.0002  0.000057   0.009579   
22          0.0095    0.0000   0.0001  0.0001  0.0001  0.000071   0.009814   
23          0.0096    0.0000   0.0000  0.0000  0.0000  0.000071   0.009814   

    lowerband  in_uptrend  
21   0.009379       False  
22   0.009386        True  
23   0.009386        True  

Fetching new bars for 2021-09-13T08:32:23.372784
Analyzing incoming data...
             timestamp    open    high     low   close   volume  \
21 2021-09-13 15:29:00  0.0096  0.0096  0.0095  0.0095  51149.0   
22 2021-09-13 15:30


Fetching new bars for 2021-09-13T08:34:10.096007
Analyzing incoming data...
             timestamp    open    high     low   close   volume  \
21 2021-09-13 15:31:00  0.0096  0.0096  0.0096  0.0096  26954.0   
22 2021-09-13 15:32:00  0.0096  0.0096  0.0096  0.0096   5102.0   
23 2021-09-13 15:33:00  0.0096  0.0096  0.0096  0.0096      0.0   

    previous_close  high-low  high-pc  low-pc   tr       atr  upperband  \
21          0.0096       0.0      0.0     0.0  0.0  0.000071   0.009814   
22          0.0096       0.0      0.0     0.0  0.0  0.000071   0.009814   
23          0.0096       0.0      0.0     0.0  0.0  0.000071   0.009814   

    lowerband  in_uptrend  
21   0.009386        True  
22   0.009386        True  
23   0.009386        True  

Fetching new bars for 2021-09-13T08:34:25.289730
Analyzing incoming data...
             timestamp    open    high     low   close   volume  \
21 2021-09-13 15:31:00  0.0096  0.0096  0.0096  0.0096  26954.0   
22 2021-09-13 15:32:00  0.0096

Analyzing incoming data...
             timestamp    open    high     low   close   volume  \
21 2021-09-13 15:32:00  0.0096  0.0096  0.0096  0.0096   5102.0   
22 2021-09-13 15:33:00  0.0096  0.0096  0.0096  0.0096      0.0   
23 2021-09-13 15:34:00  0.0096  0.0096  0.0096  0.0096  48074.0   

    previous_close  high-low  high-pc  low-pc   tr       atr  upperband  \
21          0.0096       0.0      0.0     0.0  0.0  0.000071   0.009814   
22          0.0096       0.0      0.0     0.0  0.0  0.000071   0.009814   
23          0.0096       0.0      0.0     0.0  0.0  0.000057   0.009771   

    lowerband  in_uptrend  
21   0.009386        True  
22   0.009386        True  
23   0.009429        True  


Fetching new bars for 2021-09-13T08:36:12.306458
Analyzing incoming data...
             timestamp    open    high     low   close   volume  \
21 2021-09-13 15:33:00  0.0096  0.0096  0.0096  0.0096      0.0   
22 2021-09-13 15:34:00  0.0096  0.0096  0.0096  0.0096  48074.0   
23 2021-09-1

Fetching new bars for 2021-09-13T08:37:59.018273
Analyzing incoming data...
             timestamp    open    high     low   close   volume  \
21 2021-09-13 15:34:00  0.0096  0.0096  0.0096  0.0096  48074.0   
22 2021-09-13 15:35:00  0.0096  0.0096  0.0096  0.0096      0.0   
23 2021-09-13 15:36:00  0.0096  0.0096  0.0096  0.0096   9567.0   

    previous_close  high-low  high-pc  low-pc   tr       atr  upperband  \
21          0.0096       0.0      0.0     0.0  0.0  0.000057   0.009771   
22          0.0096       0.0      0.0     0.0  0.0  0.000043   0.009729   
23          0.0096       0.0      0.0     0.0  0.0  0.000014   0.009643   

    lowerband  in_uptrend  
21   0.009429        True  
22   0.009471        True  
23   0.009557        True  

Fetching new bars for 2021-09-13T08:37:59.158682
Analyzing incoming data...
             timestamp    open    high     low   close   volume  \
21 2021-09-13 15:34:00  0.0096  0.0096  0.0096  0.0096  48074.0   
22 2021-09-13 15:35:00  0.0096 

...

Fetching new bars for 2021-09-13T09:07:31.124200
Analyzing incoming data...
             timestamp    open    high     low   close   volume  \
21 2021-09-13 16:04:00  0.0096  0.0096  0.0096  0.0096  10546.0   
22 2021-09-13 16:05:00  0.0096  0.0096  0.0096  0.0096  16802.0   
23 2021-09-13 16:06:00  0.0096  0.0096  0.0096  0.0096  72464.0   

    previous_close  high-low  high-pc  low-pc   tr       atr  upperband  \
21          0.0096       0.0      0.0     0.0  0.0  0.000043   0.009729   
22          0.0096       0.0      0.0     0.0  0.0  0.000029   0.009686   
23          0.0096       0.0      0.0     0.0  0.0  0.000029   0.009686   

    lowerband  in_uptrend  
21   0.009471        True  
22   0.009514        True  
23   0.009514        True  

Fetching new bars for 2021-09-13T09:07:32.318907
Analyzing incoming data...
             timestamp    open    high     low   close   volume  \
21 2021-09-13 16:04:00  0.0096  0.0096  0.0096  0.0096  10546.0   
22 2021-09-13 16:05:00  0.0096 


Fetching new bars for 2021-09-13T09:09:19.100293
Analyzing incoming data...
             timestamp    open    high     low   close   volume  \
21 2021-09-13 16:06:00  0.0096  0.0096  0.0096  0.0096  72464.0   
22 2021-09-13 16:07:00  0.0096  0.0096  0.0096  0.0096      0.0   
23 2021-09-13 16:08:00  0.0096  0.0096  0.0096  0.0096   5574.0   

    previous_close  high-low  high-pc  low-pc   tr       atr  upperband  \
21          0.0096       0.0      0.0     0.0  0.0  0.000029   0.009686   
22          0.0096       0.0      0.0     0.0  0.0  0.000029   0.009686   
23          0.0096       0.0      0.0     0.0  0.0  0.000029   0.009686   

    lowerband  in_uptrend  
21   0.009514        True  
22   0.009514        True  
23   0.009514        True  

Fetching new bars for 2021-09-13T09:09:33.265502
Analyzing incoming data...
             timestamp    open    high     low   close   volume  \
21 2021-09-13 16:06:00  0.0096  0.0096  0.0096  0.0096  72464.0   
22 2021-09-13 16:07:00  0.0096


Fetching new bars for 2021-09-13T09:11:06.084442
Analyzing incoming data...
             timestamp    open    high     low   close  volume  \
21 2021-09-13 16:07:00  0.0096  0.0096  0.0096  0.0096     0.0   
22 2021-09-13 16:08:00  0.0096  0.0096  0.0096  0.0096  5574.0   
23 2021-09-13 16:09:00  0.0096  0.0096  0.0096  0.0096     0.0   

    previous_close  high-low  high-pc  low-pc   tr       atr  upperband  \
21          0.0096       0.0      0.0     0.0  0.0  0.000029   0.009686   
22          0.0096       0.0      0.0     0.0  0.0  0.000029   0.009686   
23          0.0096       0.0      0.0     0.0  0.0  0.000014   0.009643   

    lowerband  in_uptrend  
21   0.009514        True  
22   0.009514        True  
23   0.009557        True  


Fetching new bars for 2021-09-13T09:11:21.261024
Analyzing incoming data...
             timestamp    open    high     low   close    volume  \
21 2021-09-13 16:08:00  0.0096  0.0096  0.0096  0.0096    5574.0   
22 2021-09-13 16:09:00  0.0096 


Fetching new bars for 2021-09-13T09:12:53.812841
Analyzing incoming data...
             timestamp    open    high     low   close    volume  \
21 2021-09-13 16:09:00  0.0096  0.0096  0.0096  0.0096       0.0   
22 2021-09-13 16:10:00  0.0096  0.0096  0.0095  0.0095  111360.0   
23 2021-09-13 16:11:00  0.0095  0.0095  0.0095  0.0095       0.0   

    previous_close  high-low  high-pc  low-pc      tr       atr  upperband  \
21          0.0096    0.0000      0.0  0.0000  0.0000  0.000014   0.009643   
22          0.0096    0.0001      0.0  0.0001  0.0001  0.000014   0.009593   
23          0.0095    0.0000      0.0  0.0000  0.0000  0.000014   0.009543   

    lowerband  in_uptrend  
21   0.009557        True  
22   0.009507       False  
23   0.009457       False  

Fetching new bars for 2021-09-13T09:13:07.979691
Analyzing incoming data...
             timestamp    open    high     low   close    volume  \
21 2021-09-13 16:09:00  0.0096  0.0096  0.0096  0.0096       0.0   
22 2021-09-1

KeyboardInterrupt: 

In [ ]:
#Keep running...

#Needs: 
#visualization
#balance tracker

In [ ]:
df = pd.DataFrame(exchange.fetch_balance()['info']['balances'])
df['free'] = pd.to_numeric(df['free'])
df['locked'] = pd.to_numeric(df['locked'])
df = df[df.free!=0].drop(columns='locked').reset_index(drop=True)

In [151]:
bars = exchange.fetch_ohlcv('VTHO/USD', timeframe='1m', limit=5)
int(10/bars[4][1]-(.05*(10/bars[4][1])))

1000

In [120]:
exchange.create_market_buy_order('VTHO/USD',1100)

{'info': {'symbol': 'VTHOUSD',
  'orderId': '21356239',
  'orderListId': '-1',
  'clientOrderId': 'x-R4BD3S82c4c4b3abd4e71dc009d278',
  'transactTime': '1631518396580',
  'price': '0.0000',
  'origQty': '1100.00000000',
  'executedQty': '1100.00000000',
  'cummulativeQuoteQty': '10.4500',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'fills': [{'price': '0.0095',
    'qty': '1100.00000000',
    'commission': '0.00001992',
    'commissionAsset': 'BNB',
    'tradeId': '4227218'}]},
 'id': '21356239',
 'clientOrderId': 'x-R4BD3S82c4c4b3abd4e71dc009d278',
 'timestamp': 1631518396580,
 'datetime': '2021-09-13T07:33:16.580Z',
 'lastTradeTimestamp': None,
 'symbol': 'VTHO/USD',
 'type': 'market',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'buy',
 'price': 0.0095,
 'stopPrice': None,
 'amount': 1100.0,
 'cost': 10.45,
 'average': 0.0095,
 'filled': 1100.0,
 'remaining': 0.0,
 'status': 'closed',
 'fee': {'cost': 1.992e-05, 'currency': 'BNB'},
 't

In [121]:
order = {'info': {'symbol': 'VTHOUSD',
  'orderId': '21356239',
  'orderListId': '-1',
  'clientOrderId': 'x-R4BD3S82c4c4b3abd4e71dc009d278',
  'transactTime': '1631518396580',
  'price': '0.0000',
  'origQty': '1100.00000000',
  'executedQty': '1100.00000000',
  'cummulativeQuoteQty': '10.4500',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'fills': [{'price': '0.0095',
    'qty': '1100.00000000',
    'commission': '0.00001992',
    'commissionAsset': 'BNB',
    'tradeId': '4227218'}]},
 'id': '21356239',
 'clientOrderId': 'x-R4BD3S82c4c4b3abd4e71dc009d278',
 'timestamp': 1631518396580,
 'datetime': '2021-09-13T07:33:16.580Z',
 'lastTradeTimestamp': None,
 'symbol': 'VTHO/USD',
 'type': 'market',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'buy',
 'price': 0.0095,
 'stopPrice': None,
 'amount': 1100.0,
 'cost': 10.45,
 'average': 0.0095,
 'filled': 1100.0,
 'remaining': 0.0,
 'status': 'closed',
 'fee': {'cost': 1.992e-05, 'currency': 'BNB'},
 'trades': [{'info': {'price': '0.0095',
    'qty': '1100.00000000',
    'commission': '0.00001992',
    'commissionAsset': 'BNB',
    'tradeId': '4227218'},
   'timestamp': None,
   'datetime': None,
   'symbol': 'VTHO/USD',
   'id': None,
   'order': None,
   'type': None,
   'side': None,
   'takerOrMaker': None,
   'price': 0.0095,
   'amount': 1100.0,
   'cost': 10.45,
   'fee': {'cost': 1.992e-05, 'currency': 'BNB'}}],
 'fees': [{'cost': 1.992e-05, 'currency': 'BNB'}]}

In [146]:
#Order details. printable.
order['info']['symbol'],order['info']['orderId'], order['info']['status'], order['trades'][0]['info']['price'], order['info']['executedQty']

('VTHOUSD', '21356239', 'FILLED', '0.0095', '1100.00000000')

In [145]:
order['trades'][0]['info']['price']

'0.0095'

In [17]:
exchange.fetch_ohlcv("DOGE/USD", timeframe='1m', limit=100)

[[1631562660000, 0.2357, 0.2357, 0.2357, 0.2357, 8666.0],
 [1631562720000, 0.2357, 0.2357, 0.2356, 0.2357, 51516.0],
 [1631562780000, 0.2357, 0.2357, 0.2357, 0.2357, 1030.0],
 [1631562840000, 0.2355, 0.2355, 0.2352, 0.2353, 45766.0],
 [1631562900000, 0.2354, 0.2355, 0.2354, 0.2354, 6585.0],
 [1631562960000, 0.2354, 0.2356, 0.2354, 0.2356, 1789.0],
 [1631563020000, 0.2358, 0.2359, 0.2358, 0.2359, 841.0],
 [1631563080000, 0.2358, 0.2358, 0.2358, 0.2358, 1548.0],
 [1631563140000, 0.2359, 0.236, 0.2358, 0.236, 10727.0],
 [1631563200000, 0.236, 0.2362, 0.236, 0.2362, 6203.0],
 [1631563260000, 0.2362, 0.2364, 0.2362, 0.2364, 23741.0],
 [1631563320000, 0.2365, 0.2367, 0.2364, 0.2366, 134006.0],
 [1631563380000, 0.2367, 0.2369, 0.2366, 0.2369, 36477.0],
 [1631563440000, 0.2368, 0.2368, 0.2367, 0.2367, 68663.0],
 [1631563500000, 0.2369, 0.237, 0.2367, 0.237, 49863.0],
 [1631563560000, 0.237, 0.2372, 0.2369, 0.2369, 58834.0],
 [1631563620000, 0.2368, 0.2371, 0.2368, 0.2369, 7180.0],
 [1631563680

In [11]:

import pytz
datetime.now().isoformat().replace(tzinfo=pytz.utc)

TypeError: replace() takes no keyword arguments